# Survival Prediction Models

This notebook compares the Random Survival Forest model to the Cox Proportional Hazards model. We have examined the assumptions for this statistical model in the notebook: http://localhost:8888/lab/tree/A/2023-ca4021-bradyd-35-mcdaida-3/nbs/Proportional_Hazards_Assumptions.ipynb

Survival models require two outcome variables - an event variable and a time to event variable. We are using 'GraftSurvivalDays' and 'GraftCensored'.

## Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Random Survival Forest
###https://pypi.org/project/random-survival-forest/
## pip install random-survival-forest
from random_survival_forest.models import RandomSurvivalForest
from random_survival_forest.scoring import concordance_index

from sklearn import set_config
from sklearn.model_selection import train_test_split

### Cox proportional Hazards
from lifelines import CoxPHFitter
from statsmodels.stats.outliers_influence import variance_inflation_factor
from lifelines import KaplanMeierFitter

## Preparing Data

In [ ]:
df_pheno = pd.read_pickle('../data/proc/pheno_eng.pkl')
df_pheno.columns

Index(['RecId', 'DonId', 'GraftSurvivalDays', 'GraftCensored', 'RecAge',
       'DonAge', 'RecSex', 'DonSex', 'GraftNo', 'PrimaryRenalDisease',
       'HasDiabetes', 'eGFR1Year', 'eGFR5Year', 'RecPC1', 'RecPC2', 'RecPC3',
       'DonPC1', 'DonPC2', 'DonPC3', 'RecHypertensionPRS',
       'DonHypertensionPRS', 'RecAlbuminuriaPRS', 'DonAlbuminuriaPRS',
       'ReceGFRPRS', 'DoneGFRPRS', 'ReceGFRDeltaPRS', 'DoneGFRDeltaPRS',
       'RecStrokePRS', 'DonStrokePRS', 'RecIAPRS', 'DonIAPRS', 'RecHAKVPRS',
       'DonHAKVPRS', 'RecPKDPRS', 'DonPKDPRS', 'RecKVPRS', 'DonKVPRS',
       'GraftDate', 'GraftType', 'OnDialysis',
       'HasPregnancyInducedHypertension', 'DonType', 'HLAMismatches',
       'ColdIschemiaTime', 'RecSex_num', 'DonSex_num',
       'PrimaryRenalDisease_num', 'GraftType_num', 'AgeDifference',
       'SexMismatch', 'Year', 'Month', 'Day', 'Season', 'Season_num',
       'DonAge_sqrd', 'RecAge_sqrd', 'HLAMismatches_sqrd',
       'AgeDifference_sqrd', 'DonAge_X_RecAge'],
      dty

In [ ]:
df_pheno['PrimaryRenalDisease_num'].head()

0    224.0
1    212.0
2    241.0
3    282.0
4    212.0
Name: PrimaryRenalDisease_num, dtype: category
Categories (54, float64): [224.0, 212.0, 241.0, 282.0, ..., 239.0, 296.0, 292.0, 279.0]

In [ ]:
# Converting has diabetes column from boolean to 1 (true) and 0 (false)
df_pheno["HasDiabetes"] = df_pheno["HasDiabetes"].astype(int)

# Converting PIH column from boolean to 1 (true) and 0 (false)
df_pheno['HasPregnancyInducedHypertension'] = df_pheno['HasPregnancyInducedHypertension'].astype(int)

# Carrying out integer encoding on the Graft type categorical data. Kidney only = 0 and double kidney = 1
# Carrying out integer encoding on the Donor type categorical data. 0 = 0 and 1 = 1
Graft_nums = {"GraftType": {"Kidney only": 0, "Double kidney": 1},
              "DonType": {"0":0, "1": 1}}
clean_df = df_pheno.replace(Graft_nums)
clean_df['GraftType'] = clean_df['GraftType'].astype(int)
clean_df['DonType'] = clean_df['DonType'].astype(int)
clean_df['PrimaryRenalDisease_num'] = clean_df['PrimaryRenalDisease_num'].astype(int)

clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1878 entries, 0 to 2103
Data columns (total 60 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   RecId                            1878 non-null   object        
 1   DonId                            1878 non-null   object        
 2   GraftSurvivalDays                1878 non-null   int64         
 3   GraftCensored                    1878 non-null   int64         
 4   RecAge                           1878 non-null   int64         
 5   DonAge                           1878 non-null   int64         
 6   RecSex                           1878 non-null   category      
 7   DonSex                           1878 non-null   category      
 8   GraftNo                          1878 non-null   int64         
 9   PrimaryRenalDisease              1878 non-null   category      
 10  HasDiabetes                      1878 non-null   int64      

## Cox Proportional Hazards model

In [ ]:
cph = CoxPHFitter(alpha=0.5) ##95% confidence Interval

## Carrying out binning for donor eGFR PRS to pass assumptions
df_bins = df_pheno.copy()
df_bins['DoneGFRPRS_bins'] = pd.cut(df_bins['DoneGFRPRS'], bins=4)

## Removing highly correlated models
## Removing Donor and Recipient PKD PRS scores also.
x = df_bins.drop(['RecId', 'DonId', 'eGFR1Year', 
                   'eGFR5Year', 'GraftDate', 'Year', 'Month', 'Day', 'Season', 
                   'Season_num', 'PrimaryRenalDisease_num', 'GraftType_num', 'RecSex', 
                   'DonSex', 'PrimaryRenalDisease', 'SexMismatch', 'OnDialysis',
                   'HasPregnancyInducedHypertension', 'GraftType', 'RecAge', 'DoneGFRPRS'], axis=1)
cph.fit(x, duration_col = 'GraftSurvivalDays', event_col = 'GraftCensored', strata='DoneGFRPRS_bins')
cph.print_summary()

/users/k2252991/miniconda3/envs/db/lib/python3.8/site-packages/lifelines/utils/__init__.py:1102: ConvergenceWarning: Column(s) ['RecPC2', 'RecPC3', 'DonPC2', 'DonPC3'] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)


<lifelines.CoxPHFitter: fitted with 1878 total observations, 1497 right-censored observations>
             duration col = 'GraftSurvivalDays'
                event col = 'GraftCensored'
                   strata = DoneGFRPRS_bins
      baseline estimation = breslow
   number of observations = 1878
number of events observed = 381
   partial log-likelihood = -2129.89
         time fit was run = 2023-03-07 15:46:41 UTC

---
                     coef  exp(coef)   se(coef)   coef lower 50%   coef upper 50%  exp(coef) lower 50%  exp(coef) upper 50%
covariate                                                                                                                  
DonAge              -0.07       0.93       0.03            -0.09            -0.05                 0.91                 0.95
GraftNo              0.60       1.83       0.16             0.50             0.71                 1.65                 2.03
HasDiabetes          0.40       1.49       0.16             0.29             0.50                 1.33                 1.66
RecPC1               5.03     152.51       5.38             1.40             8.66                 4.04              5755.74
RecPC2              -4.26       0.01      15.53           -14.74             6.21                 0.00               499.22
RecPC3               8.00    2993.72      15.40            -2.38            18.39                 0.09             9.70e+07
DonPC1               6.81     911.27       5.46             3.13            10.50                22.95             36186.04
DonPC2               7.88    2652.60      15.68            -2.69            18.46                 0.07             1.04e+08
DonPC3             -22.32       0.00      14.93           -32.39           -12.25                 0.00                 0.00
RecHypertensionPRS  -0.02       0.98       0.06            -0.06             0.01                 0.94                 1.01
DonHypertensionPRS   0.00       1.00       0.06            -0.04             0.04                 0.97                 1.04
RecAlbuminuriaPRS    0.00       1.00       0.05            -0.03             0.04                 0.97                 1.04
DonAlbuminuriaPRS   -0.01       0.99       0.05            -0.05             0.03                 0.95                 1.03
ReceGFRPRS          -0.00       1.00       0.05            -0.04             0.03                 0.96                 1.03
ReceGFRDeltaPRS     -0.02       0.98       0.05            -0.06             0.01                 0.94                 1.01
DoneGFRDeltaPRS      0.03       1.03       0.05            -0.00             0.07                 1.00                 1.07
RecStrokePRS         0.04       1.04       0.05             0.00             0.08                 1.00                 1.08
DonStrokePRS        -0.07       0.93       0.05            -0.10            -0.03                 0.90                 0.97
RecIAPRS             0.00       1.00       0.05            -0.03             0.04                 0.97                 1.04
DonIAPRS             0.07       1.08       0.05             0.04             0.11                 1.04                 1.12
RecHAKVPRS           0.14       1.15       0.07             0.09             0.19                 1.09                 1.21
DonHAKVPRS           0.06       1.07       0.07             0.01             0.11                 1.01                 1.12
RecPKDPRS           -0.00       1.00       0.05            -0.04             0.03                 0.96                 1.03
DonPKDPRS            0.03       1.03       0.06            -0.01             0.07                 0.99                 1.07
RecKVPRS            -0.08       0.92       0.07            -0.13            -0.03                 0.88                 0.97
DonKVPRS            -0.08       0.93       0.07            -0.13            -0.03                 0.88                 0.97
DonType             -0.19       0.83       0.29            -0.39             0.01                 0.68

## Random Survival Forest

In [ ]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1878 entries, 0 to 2103
Data columns (total 60 columns):
 #   Column                           Non-Null Count  Dtype         
---  ------                           --------------  -----         
 0   RecId                            1878 non-null   object        
 1   DonId                            1878 non-null   object        
 2   GraftSurvivalDays                1878 non-null   int64         
 3   GraftCensored                    1878 non-null   int64         
 4   RecAge                           1878 non-null   int64         
 5   DonAge                           1878 non-null   int64         
 6   RecSex                           1878 non-null   category      
 7   DonSex                           1878 non-null   category      
 8   GraftNo                          1878 non-null   int64         
 9   PrimaryRenalDisease              1878 non-null   category      
 10  HasDiabetes                      1878 non-null   int64      

In [ ]:
x = clean_df[['RecAge', 'DonAge']]


y = df_pheno.loc[:, ['GraftSurvivalDays', 'GraftCensored']]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=123)

In [ ]:
rsf = RandomSurvivalForest(n_estimators=20, n_jobs=-1)
rsf = rsf.fit(x_train, y_train)
y_pred = rsf.predict(x_test)
c_val = concordance_index(y_time=y_test["GraftSurvivalDays"], y_pred=y_pred, y_event=y_test["GraftCensored"])
print("C-index", round(c_val, 3))